# Topic Modelling on Program Source Code
---
By Kishalay Banerjee, Dan Jones and Sam Harding

In [1]:
import warnings
warnings.filterwarnings('ignore')  # 0y

In [2]:
import pandas
import pickle
import math
import numpy
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [3]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from urllib.request import urlretrieve

In [4]:
numpy.random.seed(0xD00D5) 

## Preparation

There are a number of files which are too large to store on GitHub. These are hosted on our server, and can be downloaded by running the following cell:

In [5]:
files = [
    'full_lda_model.pickle',
    'full_tf.pickle',
    'full_tf_vectorizer.pickle',
    'full-dataset.csv.gz',
]

base_url = 'https://daniel.wilshirejones.com/private-uUX6IzfsRYLNiti4ZFmgv6U3dFInnq37r5YSQs46iejeB96q0MAy9Ko7hkgo/'
destination_directory = '../data/'

for file in files:
    url = base_url + file
    destination = destination_directory + file
    print("Downloading '{}'' to location '{}'".format(url, destination))
    urlretrieve(url, destination)

## Generating the Dataset

TODO: Import dataset.py and explain how it's used + what it does.

TODO: Add Sam's scrub function in here?

In [6]:
minimal_dataset = pandas.read_csv("../data/dataset.csv.gz", header=None, names=['repo', 'language', 'documents'])
minimal_dataset.head()

,repo,language,documents
0,28457823,javascript,"b""module.exports = {\n plugins: [\n requir..."
1,28457823,javascript,"b""// The path where to mount the REST API app\..."
2,28457823,javascript,"b""import { Observable } from 'rx';\nimport deb..."
3,28457823,javascript,"b""import { Observable } from 'rx';\n// import ..."
4,28457823,javascript,"b""import { Observable } from 'rx';\n\nmodule.e..."


In [7]:
full_dataset = pandas.read_csv("../data/full-dataset.csv.gz", header=None, names=['repo', 'language',  'topics', 'documents'])

# Remove Github 'topics' since we don't use them in this analysis
full_dataset = full_dataset.drop(columns='topics')

full_dataset.head()

,repo,language,documents
0,28457823,javascript,"b""module.exports = {\n plugins: [\n requir..."
1,28457823,javascript,"b""// The path where to mount the REST API app\..."
2,28457823,javascript,"b""import { Observable } from 'rx';\nimport deb..."
3,28457823,javascript,"b""import { Observable } from 'rx';\n// import ..."
4,28457823,javascript,"b""import { Observable } from 'rx';\n\nmodule.e..."


In [41]:
test_dataset = pandas.read_csv("../data/test-dataset.csv.gz", header=None, names=['repo', 'language', 'topics', 'documents'])

# Remove Github 'topics' since we don't use them in this analysis
test_dataset = test_dataset.drop(columns='topics')

test_dataset.head()

,repo,language,documents
0,69798748,javascript,"b""const glob = require('glob')\nconst markdown..."
1,128624453,javascript,"b""module.exports = {\n extends: ['@commitlint..."
2,128624453,javascript,"b""module.exports = {\n extends: ['standard', ..."
3,128624453,javascript,"b""const cp = require('child_process')\nconst g..."
4,128624453,javascript,"b""module.exports = {\n verbose: true,\n tran..."


To evaluate the mixture model, we must label each repository with it's percentage of each programming language:

In [42]:
def calculate_language_percentages(group):
    total_python_length = 0
    total_r_length = 0
    total_javascript_length = 0
    
    for index, repo, language, document in group.itertuples():
        if language == 'python':
            total_python_length += len(document)
            
        if language == 'javascript':
            total_javascript_length += len(document)
            
        if language == 'r':
            total_r_length += len(document)
            
    total_length = total_python_length + total_r_length + total_javascript_length
            
    return pandas.Series([
        total_python_length/total_length,
        total_r_length/total_length,
        total_javascript_length/total_length,
    ])

In [43]:
test_composition_actual = test_dataset.groupby(by='repo').apply(calculate_language_percentages)
test_composition_actual.columns = ['python', 'r', 'javascript']

Here are the programming language percentages for each of repository in our test dataset:

In [44]:
test_composition_actual

,python,r,javascript
repo,,,
596892,1.000000,0.000000,0.000000
1248263,0.652443,0.000000,0.347557
1790564,0.000000,0.000000,1.000000
4751958,0.360730,0.000000,0.639270
12465340,0.000000,0.995105,0.004895
13523710,0.000000,1.000000,0.000000
14267375,0.941701,0.000000,0.058299
14579179,0.000000,0.189083,0.810917
16146440,0.000000,0.253421,0.746579


In [45]:
combined_test_documents = test_dataset.groupby(by='repo').apply(concat_texts)

## Topic Modelling on Individual Source Files

Basically done, just need to copy it over. Maybe run on the bigger dataset?

TODO:
  - Copy work from documentation/daniel-jones.ipynb
  - Add visualisation with pyldavis

For our purposes, common words are important and rare words aren't. So we shouldn't use tf-idf as a metric, bag-of-words makes more sense. (TODO: Maybe: "Similarly, filter out words that don't occur very often").


In [8]:
documents = minimal_dataset['documents']

In [ ]:
tf_vectorizer = CountVectorizer(stop_words=None)
tf = tf_vectorizer.fit_transform(documents)

with open('../data/minimal_lda_tf.pickle', 'wb') as f:
    pickle.dump(tf, f)
    
with open('../data/minimal_lda_tf_vectorizer.pickle', 'wb') as f:
    pickle.dump(tf_vectorizer, f)

We have four programming languages, try to use LDA to determine these four programming languages.

In [ ]:
number_of_languages = 4

lda = LatentDirichletAllocation(n_topics=number_of_languages,  n_jobs=1)
model = lda.fit(tf)

with open('../data/minimal_lda_model.pickle', 'wb') as f:
    pickle.dump(model, f)

In [9]:
with open('../data/minimal_lda_model.pickle', 'rb') as f:
    model = pickle.load(f)
    
with open('../data/minimal_lda_tf.pickle', 'rb') as f:
    tf = pickle.load(f)
    
with open('../data/minimal_lda_tf_vectorizer.pickle', 'rb') as f:
    tf_vectorizer = pickle.load(f)

In [10]:
pyLDAvis.sklearn.prepare(model, tf, tf_vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.170498  0.068422       1        1  53.661148
3      0.334580 -0.137167       2        1  17.800762
1     -0.215109 -0.203095       3        1  16.993209
2      0.051028  0.271840       4        1  11.544881, topic_info=       Category          Freq           Term         Total  loglift  logprob
term                                                                        
224987  Default  76787.000000           self  76787.000000  30.0000  30.0000
209790  Default  28422.000000          react  28422.000000  29.0000  29.0000
118970  Default  35518.000000           from  35518.000000  28.0000  28.0000
257199  Default  93096.000000           this  93096.000000  27.0000  27.0000
256596  Default  71532.000000            the  71532.000000  26.0000  26.0000
138187  Default  75532.000000             if  75532.000000  25.0000  25.0000
93741   Default  26333.000000        default  26333.000000  24.0000  24.0000
93727   Default  19823.000000            def  19823.000000  23.0000  23.0000
119762  Default  78705.000000       function  78705.000000  22.0000  22.0000
139197  Default  26520.000000         import  26520.000000  21.0000  21.0000
180274  Default  18618.000000        nimport  18618.000000  20.0000  20.0000
178605  Default  13422.000000        nexport  13422.000000  19.0000  19.0000
6203    Default  19257.000000             12  19257.000000  18.0000  18.0000
179191  Default  15713.000000          nfrom  15713.000000  17.0000  17.0000
88624   Default  12159.000000  createsvgicon  12159.000000  16.0000  16.0000
195902  Default  18062.000000           path  18062.000000  15.0000  15.0000
182715  Default  21713.000000           none  21713.000000  14.0000  14.0000
270897  Default  16822.000000           user  16822.000000  13.0000  13.0000
10108   Default  14036.000000             16  14036.000000  12.0000  12.0000
216978  Default  74742.000000         return  74742.000000  11.0000  11.0000
139409  Default  34335.000000             in  34335.000000  10.0000  10.0000
273107  Default  50512.000000            var  50512.000000   9.0000   9.0000
58561   Default  16313.000000             as  16313.000000   8.0000   8.0000
118218  Default   8955.000000       fragment   8955.000000   7.0000   7.0000
4646    Default  16616.000000             10  16616.000000   6.0000   6.0000
281843  Default   7860.000000            xd0   7860.000000   5.0000   5.0000
16221   Default  10334.000000             24  10334.000000   4.0000   4.0000
80378   Default   8276.000000      classname   8276.000000   3.0000   3.0000
281885  Default   7548.000000            xe0   7548.000000   2.0000   2.0000
173909  Default  23687.000000           name  23687.000000   1.0000   1.0000
...         ...           ...            ...           ...      ...      ...
281761   Topic4   1464.391410            xbd   1487.025518   2.1436  -6.2404
281631   Topic4   1663.439660            xb4   1699.407348   2.1375  -6.1129
281809   Topic4   2187.518169            xc3   2248.312349   2.1315  -5.8390
209790   Topic4  26136.460497          react  28422.863801   2.0751  -3.3585
281786   Topic4   1213.602554            xbf   1241.477370   2.1362  -6.4282
80378    Topic4   7546.761037      classname   8276.007692   2.0667  -4.6007
281561   Topic4   1517.149330            xb0   1563.031406   2.1291  -6.2050
281345   Topic4   1061.601815            xa5   1081.515369   2.1403  -6.5620
205348   Topic4   4516.349076      proptypes   4899.945261   2.0774  -5.1141
280778   Topic4   2783.143067            x80   2961.846853   2.0967  -5.5982
281673   Topic4   2447.870929            xb8   2614.659009   2.0930  -5.7266
125631   Topic4   1890.410526        gettext   2012.120308   2.0965  -5.9850
118970   Topic4  25303.791250           from  35518.688459   1.8198  -3.3908
159703   Topic4   3531.937969             m0   4032.249447   2.0265  -5.3600
180274   Topic

Try to do this on the full dataset:

In [ ]:
number_of_languages = 4
all_documents = full_dataset['documents']

full_tf_vectorizer = CountVectorizer(stop_words=None)
full_tf = full_tf_vectorizer.fit_transform(all_documents)
full_tf_feature_names = full_tf_vectorizer.get_feature_names()

full_lda = LatentDirichletAllocation(n_topics=number_of_languages,  n_jobs=1)
full_model = full_lda.fit(full_tf)

with open('../data/full_lda_model.pickle', 'wb') as f:
    pickle.dump(full_model, f)

with open('../data/full_tf.pickle', 'wb') as f:
    pickle.dump(full_tf, f)
    
with open('../data/full_tf_vectorizer.pickle', 'wb') as f:
    pickle.dump(full_tf_vectorizer, f)

In [11]:
with open('../data/full_lda_model.pickle', 'rb') as f:
    full_model = pickle.load(f)

with open('../data/full_tf.pickle', 'rb') as f:
    full_tf = pickle.load(f)
    
with open('../data/full_tf_vectorizer.pickle', 'rb') as f:
    full_tf_vectorizer = pickle.load(f)

Visualize our new model. Note that the following code cell requires more than 8 GB of RAM to run.

In [ ]:
pyLDAvis.sklearn.prepare(full_model, full_tf, full_tf_vectorizer)

Reference: https://nbviewer.jupyter.org/github/bmabey/pyLDAvis/blob/master/notebooks/sklearn.ipynb

## Topic Modelling on Programming Language Mixtures
Keypoint: topics are programming languages, file with mixture of programming languages, identify which is which.

Applicability to cyber-security: identifying malware embedded within normal programs (shellcode).

### Combining Source Files per Repository

Currently our data set consists of one data point per file containing:
  1. ID of the Github repository the file belongs to.
  2. Programming language it is written in (identified by file extension).
  3. File contents
  
In this section, we extend our analysis from working on documents with one language per file, to a system where there is a mixture of languages inside each document. To do this, we combine all the files in each repository into a single data point.

In [12]:
def concat_texts(group):
    [repo_id] = group['repo'].unique()
    combined = ' '.join(group['documents'])
    return combined

In [13]:
combined_documents = minimal_dataset.groupby(by='repo').apply(concat_texts)

In [14]:
combined_documents.head()

repo
19438      b"#' @include ggplot-global.R\n#' @include ggp...
26554      b'# The contents of this file are subject to t...
544208     b'import logging\nimport os\nimport platform a...
643909     b'#\' Environment variables to set when callin...
2594513    b'# S3 method to deal with chunks and inline t...
dtype: object

### LDA Model 

In [51]:
combined_test_documents.append(combined_documents)

repo
596892       b'from __future__ import print_function\n\nfro...
1248263      b'// VT100.js -- a text terminal emulator in J...
1790564      b"'use strict';\n\nconst ExtractTextPlugin = r...
4751958      b'from .celery import app as celery_app\n\n__a...
12465340     b'#\' @title Aggregation object.\n#\' @descrip...
13523710     b'do_nxt <- function(e)UseMethod("do_nxt")\ndo...
14267375     b'import unittest\nimport os\nimport json\n\nf...
14579179     b'#\' Add data to a plotly visualization\n#\' ...
16146440     b'base64_encode_file <- function(in_file, enco...
17856544     b'#\' Complete list of palettes\n#\'\n#\' Use ...
19117456     b'import React, {Component} from \'react\';\ni...
21289110     b'# coding: utf-8\n\n"""\n    The approach tak...
23932217     b'\n# TODO:\n#  - catch errors and throw a war...
24929423     b'############################################...
28556914     b'#\' R + mermaid.js\n#\'\n#\' Make diagrams i...
33614304     b'# Copyright (c) 2016, Aaron Christi

In [52]:
tf_vectorizer = CountVectorizer(stop_words=None)
tf_vectorizer.fit(combined_test_documents.append(combined_documents))

tf_train = tf_vectorizer.transform(combined_documents)
tf_test = tf_vectorizer.transform(combined_test_documents)

with open('../data/concatDocs_tf_vectorizer.pickle', 'wb') as f:
    pickle.dump(tf_vectorizer, f)
    
with open('../data/concatDocs_tf_train.pickle', 'wb') as f:
    pickle.dump(tf_train, f)
    
with open('../data/concatDocs_tf_test.pickle', 'wb') as f:
    pickle.dump(tf_test, f)

In [53]:
number_of_languages = 4

lda = LatentDirichletAllocation(n_topics=number_of_languages,  n_jobs=1)
model = lda.fit(tf_train)

with open('../data/concatDocs_lda_model.pickle', 'wb') as f:
    pickle.dump(model, f)

/home/sharding/miniconda3/envs/myenv/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


In [54]:
with open('../data/concatDocs_lda_model.pickle', 'rb') as f:
    model = pickle.load(f)
    
with open('../data/concatDocs_tf_train.pickle', 'rb') as f:
    tf_train = pickle.load(f)
    
with open('../data/concatDocs_tf_vectorizer.pickle', 'rb') as f:
    tf_vectorizer = pickle.load(f)

with open('../data/concatDocs_tf_test.pickle', 'rb') as f:
    tf_test = pickle.load(f)

In [60]:
pyLDAvis.sklearn.prepare(model, tf_train, tf_vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.188152  0.041085       1        1  41.383718
0      0.190932  0.023344       2        1  35.719215
3     -0.107233 -0.180432       3        1  12.489208
2     -0.271851  0.116003       4        1  10.407859, topic_info=       Category          Freq           Term         Total  loglift  logprob
term                                                                        
272177  Default  34078.000000          react  34078.000000  30.0000  30.0000
152146  Default  37731.000000           from  37731.000000  29.0000  29.0000
9020    Default  22611.000000             12  22611.000000  28.0000  28.0000
14064   Default  16671.000000             16  16671.000000  27.0000  27.0000
113531  Default  13997.000000  createsvgicon  13997.000000  26.0000  26.0000
292282  Default  74805.000000           self  74805.000000  25.0000  25.0000
232874  Default  20292.000000        nimport  20292.000000  24.0000  24.0000
120001  Default  25929.000000        default  25929.000000  23.0000  23.0000
254582  Default  19043.000000           path  19043.000000  22.0000  22.0000
230415  Default  14609.000000        nexport  14609.000000  21.0000  21.0000
6769    Default  17529.000000             10  17529.000000  20.0000  20.0000
21337   Default  12502.000000             24  12502.000000  19.0000  19.0000
25875   Default  11129.000000             32  11129.000000  18.0000  18.0000
151322  Default  10319.000000       fragment  10319.000000  17.0000  17.0000
33748   Default  10775.000000             48  10775.000000  16.0000  16.0000
8131    Default  12159.000000             11  12159.000000  15.0000  15.0000
11483   Default  10895.000000             14  10895.000000  14.0000  14.0000
10608   Default  10344.000000             13  10344.000000  13.0000  13.0000
19830   Default   9375.000000             21   9375.000000  12.0000  12.0000
179408  Default  26448.000000         import  26448.000000  11.0000  11.0000
14863   Default   9027.000000             17   9027.000000  10.0000  10.0000
32379   Default   9031.000000             45   9031.000000   9.0000   9.0000
13353   Default   9231.000000             15   9231.000000   8.0000   8.0000
337054  Default  88849.000000           this  88849.000000   7.0000   7.0000
22480   Default   6687.000000             26   6687.000000   6.0000   6.0000
36168   Default   5433.000000            512   5433.000000   5.0000   5.0000
109063  Default  20204.000000          const  20204.000000   4.0000   4.0000
76632   Default  18887.000000         assert  18887.000000   3.0000   3.0000
20337   Default   7413.000000             22   7413.000000   2.0000   2.0000
23094   Default   6752.000000             28   6752.000000   1.0000   1.0000
...         ...           ...            ...           ...      ...      ...
21337    Topic4  10315.465417             24  12502.802671   2.0703  -4.1845
14064    Topic4  13536.019739             16  16671.864235   2.0542  -3.9128
33748    Topic4   8873.844776             48  10775.425749   2.0684  -4.3350
22480    Topic4   5638.278801             26   6687.078928   2.0920  -4.7885
9020     Topic4  17341.029180             12  22611.391599   1.9972  -3.6650
19830    Topic4   7386.806056             21   9375.969765   2.0242  -4.5184
47112    Topic4   2030.915190             80   2384.201935   2.1022  -5.8096
23094    Topic4   5195.385838             28   6752.073174   2.0005  -4.8703
8398     Topic4    845.351349            112    926.568966   2.1709  -6.6861
14863    Topic4   6581.784360             17   9027.429109   1.9466  -4.6338
40936    Topic4   3905.693888             64   5063.239099   2.0030  -5.1557
11483    Topic4   7767.994932             14  10895.408682   1.9243  -4.4681
10608    Topic4   7284.549603             13  10344.101833   1.9119  -4.5324
30367    Topic4   2847.845482             40   3605.540714   2.0267  -5.4716
20818    Topic

## Identifying Program Subjects and Themes

Hypothesis: Using tf-idf rather than bag-of-words as an input to LDA will prioritise rare words. In the case of source code, this means programming language keywords (an identifying feature of programming languages) are deprioritised, and so a more human idea of topics may emerge. 

We can use repo-list.json and the repo-ids to map the github topics/tags to each repo. Might be a small/easy task to compare against the programming langauge identification.

In [16]:
documents = minimal_dataset['documents']

In [18]:
tf_vectorizer = TfidfVectorizer(stop_words=None)
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

with open('../data/tfidf_lda_tf.pickle', 'wb') as f:
    pickle.dump(tf, f)
    
with open('../data/tfidf_lda_tf_vectorizer.pickle', 'wb') as f:
    pickle.dump(tf_vectorizer, f)

We have four programming languages, try to use LDA to determine these four programming languages.

In [19]:
number_of_themes = 3

lda = LatentDirichletAllocation(n_topics=number_of_themes,  n_jobs=1)
model = lda.fit(tf)

with open('../data/tfidf_lda_model.pickle', 'wb') as f:
    pickle.dump(model, f)

/home/sharding/miniconda3/envs/myenv/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


In [20]:
with open('../data/tfidf_lda_model.pickle', 'rb') as f:
    model = pickle.load(f)
    
with open('../data/tfidf_lda_tf.pickle', 'rb') as f:
    tf = pickle.load(f)
    
with open('../data/tfidf_lda_tf_vectorizer.pickle', 'rb') as f:
    tf_vectorizer = pickle.load(f)

In [ ]:
pyLDAvis.sklearn.prepare(model, tf, tf_vectorizer)

This still prioritises programming language keywords. One approach to solving this problem is to consider all keywords as "stopwords". First, gather a list of R, Python and Javascript keywords:

In [21]:
import keyword

python_keywords = keyword.kwlist
python_keywords

['False',
 'None',
 'True',
 'and',
 'as',
 'assert',
 'break',
 'class',
 'continue',
 'def',
 'del',
 'elif',
 'else',
 'except',
 'finally',
 'for',
 'from',
 'global',
 'if',
 'import',
 'in',
 'is',
 'lambda',
 'nonlocal',
 'not',
 'or',
 'pass',
 'raise',
 'return',
 'try',
 'while',
 'with',
 'yield']

R reserved words (sourced from the manual: https://stat.ethz.ch/R-manual/R-devel/library/base/html/Reserved.html)

In [22]:
r_keywords = [
    "if", 
    "else", 
    "repeat",
    "while",
    "function", 
    "for",
    "in",
    "next",
    "break",
    "TRUE",
    "FALSE",
    "NULL", 
    "Inf", 
    "NaN",
    "NA",
    "NA_integer_",
    "NA_real_",
    "NA_complex_",
    "NA_character_", 
]

Javascript keywords and reserved words (source: https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Lexical_grammar#Keywords)

In [23]:
javascript_keywords = [  # jaccard("ideal javascript topic", topic_i)
    "break",
    "case",
    "catch",
    "class",
    "const",
    "continue",
    "debugger",
    "default",
    "delete",
    "do",
    "else",
    "export",
    "extends",
    "finally",
    "for",
    "function",
    "if",
    "import",
    "in",
    "instanceof",
    "new",
    "return",
    "super",
    "switch",
    "this",
    "throw",
    "try",
    "typeof",
    "var",
    "void",
    "while",
    "with",
    "yield",
]

In [24]:
documents = minimal_dataset['documents']

In [28]:
tf_vectorizer = TfidfVectorizer(stop_words=javascript_keywords+python_keywords+r_keywords)
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()
    
with open('../data/tfidf_lda_tf_ignore_keywords.pickle', 'wb') as f:
    pickle.dump(tf, f)
    
with open('../data/tfidf_lda_tf_vectorizer_ignore_keywords.pickle', 'wb') as f:
    pickle.dump(tf_vectorizer, f)

We have four programming languages, try to use LDA to determine these four programming languages.

In [29]:
number_of_themes = 3

lda = LatentDirichletAllocation(n_topics=number_of_themes,  n_jobs=1)
model = lda.fit(tf)

with open('../data/tfidf_lda_model_ignore_keywords.pickle', 'wb') as f:
    pickle.dump(model, f)

/home/sharding/miniconda3/envs/myenv/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


In [30]:
with open('../data/tfidf_lda_model_ignore_keywords.pickle', 'rb') as f:
    model = pickle.load(f)
    
with open('../data/tfidf_lda_tf_ignore_keywords.pickle', 'rb') as f:
    tf = pickle.load(f)
    
with open('../data/tfidf_lda_tf_vectorizer_ignore_keywords.pickle', 'rb') as f:
    tf_vectorizer = pickle.load(f)

In [ ]:
pyLDAvis.sklearn.prepare(model, tf, tf_vectorizer)

## Measuring the Efficacy of Topic Models (WRITEUP: WHO?)

Main question: How do we evaluate how well a topic (from LDA for example) represents a meaningful "topic" or theme?

TODO: do some research on this??? There must be some papers etc that try to formalise this that we can borrow ideas from?

Paper dump:
  - Looks like a good summary paper: http://www.aclweb.org/anthology/E14-4005 Find more papers from this ones references?
    - " KL-divergence (Li and McCallum, 2006; Wang et al., 2009; Newman et al., 2009), cosine measure (He et al., 2009; Ramage et al., 2009) and the average Log Odds Ratio (Chaney and Blei, 2012). "
    - "Kim and Oh (2011) also applied  the  cosine  measure  and  KL-Divergence which were compared with four other measures: Jaccard’s Coefficient, Kendall’s τ coefficient, Discount  Cumulative  Gain  and  Jensen  Shannon  Divergence (JSD)."
  - Cool name haven't read it: http://papers.nips.cc/paper/3700-reading-tea-leaves-how-humans-interpret-topic-models.pdf
  
We considered all of these metrics, and found th Jaccard Index to be most suitable. This was primarily due to it's use of set operations, which are invariant to ordering and number of observations.

### Reading Tea Leaves Paper
http://papers.nips.cc/paper/3700-reading-tea-leaves-how-humans-interpret-topic-models.pdf

### Word Overlap

I think this is used as a baseline measure in the summary paper above (http://www.aclweb.org/anthology/E14-4005). Should be a quick implementation so worth a try.

### Jaccard Index

From the papers above this seems to have been used relatively often for _linking machine-generated topics to human topics_ and so maybe this is a good application for it. Apparently explored here "https://link.springer.com/chapter/10.1007/978-3-642-19437-5_13" but I haven't read it.


In [31]:
def jaccard_index(a, b):
    a = set(a)
    b = set(b)
    return len(a & b) / len(a | b)

Consider the group of language keywords as the best possible topic for each language. Compare each of our machine generated topics with each of our ideal topics by computing their Jaccard Index:

In [ ]:
tf_vectorizer = CountVectorizer(stop_words=None)
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

In [ ]:
len(tf_feature_names)

In [ ]:
list(enumerate(model.components_))

### Kendall’s τ Coefficient

Measures the association between two ranked lists. Source: Computational Linguistics and Intelligent Text Processing book.

### Evaluating Topic Models
TODO: better title needed

Idea:
  - save the actual % of each program langauge per repo
  - Then try to use LDA model to tell us "I believe repo <x> is 10% Topic 1, 20% Topic 2 etc". 
  - Use analysis from above two sections to create a "most likely mapping from lda topic to programming language".
  - rate our models

Here we can do cross-validation etc.

Load in the test data set:

In [32]:
test_dataset = pandas.read_csv("../data/test-dataset.csv.gz", header=None, names=['repo', 'language', 'topics', 'documents'])

# Remove Github 'topics' since we don't use them in this analysis
test_dataset = test_dataset.drop(columns='topics')

test_dataset.head()

,repo,language,documents
0,69798748,javascript,"b""const glob = require('glob')\nconst markdown..."
1,128624453,javascript,"b""module.exports = {\n extends: ['@commitlint..."
2,128624453,javascript,"b""module.exports = {\n extends: ['standard', ..."
3,128624453,javascript,"b""const cp = require('child_process')\nconst g..."
4,128624453,javascript,"b""module.exports = {\n verbose: true,\n tran..."


To evaluate the mixture model, we must label each repository with it's percentage of each programming language:

In [33]:
def calculate_language_percentages(group):
    total_python_length = 0
    total_r_length = 0
    total_javascript_length = 0
    
    for index, repo, language, document in group.itertuples():
        if language == 'python':
            total_python_length += len(document)
            
        if language == 'javascript':
            total_javascript_length += len(document)
            
        if language == 'r':
            total_r_length += len(document)
            
    total_length = total_python_length + total_r_length + total_javascript_length
            
    return pandas.Series([
        total_python_length/total_length,
        total_r_length/total_length,
        total_javascript_length/total_length,
    ])

In [34]:
test_composition_actual = test_dataset.groupby(by='repo').apply(calculate_language_percentages)
test_composition_actual.columns = ['python', 'r', 'javascript']

Here are the programming language percentages for each of repository in our test dataset:

In [36]:
test_composition_actual

,python,r,javascript
repo,,,
596892,1.000000,0.000000,0.000000
1248263,0.652443,0.000000,0.347557
1790564,0.000000,0.000000,1.000000
4751958,0.360730,0.000000,0.639270
12465340,0.000000,0.995105,0.004895
13523710,0.000000,1.000000,0.000000
14267375,0.941701,0.000000,0.058299
14579179,0.000000,0.189083,0.810917
16146440,0.000000,0.253421,0.746579


In [37]:
combined_test_documents = test_dataset.groupby(by='repo').apply(concat_texts)

In [57]:
with open('../data/concatDocs_lda_model.pickle', 'rb') as f:
    model = pickle.load(f)
    
with open('../data/concatDocs_tf_train.pickle', 'rb') as f:
    tf_train = pickle.load(f)
    
with open('../data/concatDocs_tf_vectorizer.pickle', 'rb') as f:
    tf_vectorizer = pickle.load(f)

with open('../data/concatDocs_tf_test.pickle', 'rb') as f:
    tf_test = pickle.load(f)

In [58]:
combined_test_documents_tf = tf_vectorizer.transform(combined_test_documents)
combined_test_model = model.transform(combined_test_documents_tf)

In [62]:
combined_test_model

array([[1.90661496e-01, 8.07648843e-01, 4.78215219e-06, 1.68487898e-03],
       [5.41157197e-01, 4.47795188e-01, 1.04497870e-02, 5.97828094e-04],
       [7.65079147e-01, 2.07808158e-01, 9.78836147e-03, 1.73243342e-02],
       [1.59282454e-05, 9.98934927e-01, 1.04864905e-03, 4.95710834e-07],
       [2.65019130e-01, 6.41208301e-01, 3.48327547e-03, 9.02892938e-02],
       [6.67248762e-01, 3.31216180e-01, 1.54776206e-05, 1.51958084e-03],
       [3.52752118e-04, 9.98974555e-01, 3.34400158e-04, 3.38292524e-04],
       [9.73834351e-01, 7.09203679e-07, 2.61642358e-02, 7.04124487e-07],
       [9.25075962e-01, 7.31192529e-02, 1.69474412e-06, 1.80309002e-03],
       [2.97893274e-01, 4.74194950e-01, 6.22969301e-04, 2.27288807e-01],
       [8.04771825e-01, 4.58890587e-02, 1.64098727e-05, 1.49322707e-01],
       [4.84929996e-01, 5.13540584e-01, 7.57352821e-04, 7.72067435e-04],
       [2.58767113e-01, 7.09967336e-01, 2.49810251e-03, 2.87674476e-02],
       [2.80705528e-01, 6.62976682e-01, 2.37653687e

In [ ]:
# insert kish's mappings here


### KL Divergence

In [ ]:
test_composition_estim=test_composition_actual

In [ ]:
if all(test_composition_estim.index==test_composition_actual.index)==True:
    KDDataframe=pandas.DataFrame([],columns=["Repo","KD Divergence"])
    for i in range(0,len(test_composition_actual)):
        a=0
        for j in range(0,len(test_composition_actual.iloc[i])):
            a=a-((test_composition_actual.iloc[i][j])*math.log(test_composition_actual.iloc[i][j]/test_composition_estim.iloc[i][j]))
        #m=pandas.DataFrame([test_composition_actual.index[i],a],columns=["Repo","KD Divergence"])
        print(a)
        #KDDataframe.append(m)
#KDDataframe

# Notes:
  - our keyword lists have simliar/common words e.g. python javascript and r all share some keywords. this can be seen in the documents. choosing another language, with a completely different set of keywords might prove easier to differentiate for the LDA model. somethind somthing LDA uses distance, but if true topics share key words, then distance metric breaks.